In [1]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !uv pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !uv pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !uv pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !uv pip install --no-deps unsloth

Using Python 3.11.12 environment at: /usr
Resolved 8 packages in 56ms
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
unsloth-zoo ------------------------------ 16.00 KiB/134.89 KiB
⠙ Preparing packages... (0/5)
unsloth-zoo ------------------------------ 16.00 KiB/134.89 KiB
trl        ------------------------------     0 B/311.46 KiB
⠙ Preparing packages... (0/5)
unsloth-zoo ------------------------------ 32.00 KiB/134.89 KiB
trl        ------------------------------     0 B/311.46 KiB
⠙ Preparing packages... (0/5)
unsloth-zoo ------------------------------ 48.00 KiB/134.89 KiB
trl        ------------------------------     0 B/311.46 KiB
⠙ Preparing packages... (0/5)
unsloth-zoo ------------------------------ 48.00 KiB/134.89 KiB
trl        ------------------------------ 16.00 KiB/311.46 KiB
⠙ Preparing packages... (0/5)
unsloth-zoo ------------------------------ 63.86 KiB/134.89 KiB
trl        ------------------------------ 16.00 KiB/311.46 K

In [2]:
!wget https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar
!tar -xf abo-images-small.tar

--2025-05-18 15:46:47--  https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar
Resolving amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)... 52.216.216.193, 52.217.87.92, 16.182.99.145, ...
Connecting to amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)|52.216.216.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3253381120 (3.0G) [application/x-tar]
Saving to: ‘abo-images-small.tar’

abo-images-small.ta 100%[===================>]   3.03G  16.3MB/s    in 3m 13s  

2025-05-18 15:50:01 (16.1 MB/s) - ‘abo-images-small.tar’ saved [3253381120/3253381120]



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# import os
# os.chdir('/workspace')
import torch
# ========= CONFIG =========
listing = "0"
csv_path = "/content/drive/MyDrive/images/VQA_dataset_train/merged_listings.csv"  # CSV must contain: image_path, question, one_word_answer
#get length of csv filevvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
f = open(csv_path, "r")
lines = f.readlines()
f.close()

In [5]:
model_dir = "/content/drive/MyDrive/images/model"
output_dir = "/content/drive/MyDrive/images/lora_on_listing_for_qwen2"
images_root = "dataset/abo-images-small/images/small"
batch_size = 32
gradient_accumulation_steps = 1
lenght_of_dataset = len(lines) -1
max_steps = (len(lines)-1)//(batch_size * gradient_accumulation_steps)
print(f"Steps per epoch: {max_steps}")
checkpoint_path = None
# ===========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
del lines


Steps per epoch: 38037


In [6]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch
from transformers import AutoProcessor


model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 4,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 4,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/572 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Unsloth: Making `model.base_model.model.visual` require gradients


In [7]:
from torch.utils.data import IterableDataset, DataLoader
from PIL import Image
class VQALazyIterableDataset(IterableDataset):
    def __init__(self, dataset_stream, lenght_of_dataset,tokenizer,padding_token_ids):
        self.dataset_stream = dataset_stream
        self.lenght_of_dataset = lenght_of_dataset
        self.images_root = "images/small"
        self.tokenizer = tokenizer
        self.padding_token_ids =  padding_token_ids

    def preprocess(self, example):


        image = Image.open(f"{self.images_root}/{example['image_path']}").convert("RGB")
        image = image.resize((224, 224))
        question = example["question"]
        answer = example["answer"]


        messages = [
            {"role": "user", "content": [
                {"type": "image"},
                {"type": "text", "text": f"{question}. Answer with one word."}
            ]},
            {"role": "assistant", "content": [
                {"type": "text", "text": f"{answer}"}
            ]}
        ]
        input_text = self.tokenizer.apply_chat_template(messages, add_generation_prompt = False,tokenize = False)
        inputs = self.tokenizer(
            image,
            input_text,
            add_special_tokens = False,
            max_length=300,
            truncation=True,
            return_tensors = "pt",
            padding="max_length"
        )

        labels = inputs["input_ids"].clone()
        labels[torch.isin(labels, self.padding_token_ids)] = -100



        return {
            "pixel_values": inputs["pixel_values"].squeeze(0),
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": labels.squeeze(0),
            "image_grid_thw": inputs["image_grid_thw"].squeeze(0)
            }



    def __iter__(self):
        for example in self.dataset_stream:
            processed = self.preprocess(example)
            if processed:
                yield processed
    def __len__(self):
      return self.lenght_of_dataset


In [8]:
from unsloth.trainer import UnslothVisionDataCollator
data_collator = UnslothVisionDataCollator(model, tokenizer)
def collate_fn(batch):
    import torch

    # Extract fields from batch
    pixel_values = [example["pixel_values"] for example in batch]
    input_ids = [example["input_ids"] for example in batch]
    attention_mask = [example["attention_mask"] for example in batch]
    labels = [example["labels"] for example in batch]
    image_grid_thw = [example["image_grid_thw"] for example in batch]

    # Stack vision-related tensors
    pixel_values = torch.stack(pixel_values)         # [B, 256, 1176]
    image_grid_thw = torch.stack(image_grid_thw)     # [B, 3]

    # Tokenizer padding for input_ids & attention_mask
    text_inputs = [
        {"input_ids": ids, "attention_mask": mask}
        for ids, mask in zip(input_ids, attention_mask)
    ]
    padded_inputs = tokenizer.tokenizer.pad(
        text_inputs,
        return_tensors="pt"
    )

    # Tokenizer padding for labels
    padded_labels = tokenizer.tokenizer.pad(
        [{"input_ids": lbl} for lbl in labels],
        return_tensors="pt"
    )["input_ids"]


    return {
        "pixel_values": pixel_values,                           # [B, 256, 1176]
        "input_ids": padded_inputs["input_ids"],                # [B, T]
        "attention_mask": padded_inputs["attention_mask"],      # [B, T]
        "labels": padded_labels,                                # [B, T] with -100
        "image_grid_thw": image_grid_thw                        # [B, 3]
    }


Unsloth: Model does not have a default image size - using 512


In [9]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files=csv_path, split="train", streaming=True)
dataset = dataset.filter(lambda x: x['image_path'] and x["question"] and x["answer"])

"""Let's take an overview look at the dataset. We shall see what the 3rd image is, and what caption it had."""



vqa_dataset = VQALazyIterableDataset(dataset,lenght_of_dataset, tokenizer, data_collator.padding_token_ids)

dataloader = DataLoader(
    vqa_dataset,
    batch_size=1,
)
batch = next(iter(dataloader))
for k, v in batch.items():
    print(k, v.shape)

from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
from transformers import Trainer, TrainingArguments

FastVisionModel.for_training(model) # Enable for training!


pixel_values torch.Size([1, 256, 1176])
input_ids torch.Size([1, 300])
attention_mask torch.Size([1, 300])
labels torch.Size([1, 300])
image_grid_thw torch.Size([1, 3])


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2VLForConditionalGeneration(
      (visual): Qwen2VisionTransformerPretrainedModel(
        (patch_embed): PatchEmbed(
          (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
        (rotary_pos_emb): VisionRotaryEmbedding()
        (blocks): ModuleList(
          (0-31): 32 x Qwen2VLVisionBlock(
            (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
            (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
            (attn): VisionSdpaAttention(
              (qkv): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1280, out_features=3840, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1280, out_features=4, bias=False)
                )
                (lora_B): Modul

In [ ]:
trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = collate_fn, # Must use!
    train_dataset = vqa_dataset,
    args = TrainingArguments(
        per_device_train_batch_size = batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 32,
        save_steps=32,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        report_to = "none",     # For Weights and Biases
        save_total_limit=2,
        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        max_steps = max_steps


    ),
)
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

checkpoint_path = '/content/drive/MyDrive/images/model/qwen_2.5_low_rank_checkpoints/checkpoint-9824'

trainer_stats = trainer.train(resume_from_checkpoint=checkpoint_path)

model.save_pretrained(f"{model_dir}/lora_model")  # Local saving
tokenizer.save_pretrained(f"{model_dir}/lora_model")

<ipython-input-11-2868d04dc2a9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
2.391 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,217,194 | Num Epochs = 1 | Total steps = 38,037
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 1 x 1) = 32
 "-____-"     Trainable parameters = 7,237,632/2,000,000,000 (0.36% trained)
